In [ ]:
# 0.1. Cài đặt các thư viện cần thiết
!pip install -q fpdf2 noisereduce librosa tensorflow scikit-learn matplotlib seaborn pytz PyDrive2 

In [ ]:
# CÀI ĐẶT & CẤU HÌNH (SETUP & CONFIGURATION)
# 0.2. Import thư viện
import os
import glob
import random
import datetime
import pytz
import shutil
import joblib
import zipfile
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from fpdf import FPDF
from tqdm import tqdm
import librosa
import noisereduce as nr
import tensorflow as tf
from tensorflow.keras import mixed_precision
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D, AveragePooling2D
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import StratifiedGroupKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from kaggle_secrets import UserSecretsClient
from oauth2client.service_account import ServiceAccountCredentials
from tensorflow.keras.regularizers import l2
from kaggle_datasets import KaggleDatasets
from sklearn.metrics import roc_curve, auc as sklearn_auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
mixed_precision.set_global_policy('mixed_bfloat16')

try:
    # Cố gắng kết nối với TPU bằng cách chỉ định tpu='local'
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
    print('Đã tìm thấy TPU Resolver.')
    
    # Kết nối và khởi tạo hệ thống TPU
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    print('Đã khởi tạo hệ thống TPU.')

    # Tạo strategy
    strategy = tf.distribute.TPUStrategy(tpu)
    print('THÀNH CÔNG: Đã tạo TPUStrategy!')
    print(f'Số lượng nhân (replicas): {strategy.num_replicas_in_sync}')
    
except (ValueError, RuntimeError) as e:
    # Nếu vẫn không tìm thấy TPU, tự động chuyển về chiến lược mặc định
    print(f'Lỗi kết nối TPU: {e}')
    print('Không tìm thấy TPU. Sử dụng chiến lược mặc định cho GPU/CPU.')
    strategy = tf.distribute.get_strategy()

In [ ]:
# THIẾT LẬP CẤU HÌNH 
# --- Các cấu hình cơ bản ---
SEED = 42
def set_seed(seed_value):
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    #os.environ['TF_DETERMINISTIC_OPS'] = '1'
    random.seed(seed_value)
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
set_seed(SEED)

KAGGLE_PROCESSED_DATA_PATH = "/kaggle/input/ngt-spectrogram-id/"
KAGGLE_OUTPUT_PATH = "/kaggle/working/output_results"
os.makedirs(KAGGLE_OUTPUT_PATH, exist_ok=True)

CLASSES_TO_TRAIN = ['covid', 'asthma', 'healthy', 'tuberculosis']
ALL_CLASSES = ['healthy', 'asthma', 'covid', 'tuberculosis']
N_SPLITS = 5
TEST_SPLIT_RATIO = 0.15
USE_DATA_AUGMENTATION = True # Bật/tắt augmentation ở đây
USE_FOCAL_LOSS = True

MODEL_ID = f'EfficienetB0_CV_TPU'
MIN_DELTA = 1e-4
SHUFFLE_BUFFER_SIZE = 2048 
GAMMA = 2.0 # Giữ nguyên giá trị tiêu chuẩn

# --- CÁC THAY ĐỔI CHÍNH ---
LEARNING_RATE = 5e-6              # Thay đổi: Giảm LR để ổn định hơn
WEIGHT_DECAY = 3e-3               # Thay đổi: Tăng để chống overfitting mạnh hơn

# Cấu hình cho Cross-Validation
TOTAL_EPOCHS = 180                # Giữ nguyên tổng số epochs tối đa
WARMUP_EPOCHS = 3                 # Thay đổi: Rút ngắn Warmup cho hiệu quả
RESTART_CYCLE_1_EPOCHS = 20       # Thay đổi: Chu kỳ ngắn hơn để khám phá nhiều hơn
PATIENCE_EPOCHS = RESTART_CYCLE_1_EPOCHS + 10 # Thay đổi: Patience = 50

# --- ĐỊNH NGHĨA BATCH SIZE ---
# BATCH_SIZE này là batch size cho mỗi nhân TPU (per-replica)
BATCH_SIZE = 32
# Tính toán GLOBAL_BATCH_SIZE để dùng trong pipeline
# Biến 'strategy' được lấy từ ô code đầu tiên
GLOBAL_BATCH_SIZE = BATCH_SIZE * strategy.num_replicas_in_sync
print(f"Batch size mỗi nhân: {BATCH_SIZE}")
print(f"Global batch size (tổng cộng): {GLOBAL_BATCH_SIZE}")

# INPUT_SHAPE sẽ được cập nhật lại ở ô chuẩn bị dữ liệu
INPUT_SHAPE = (224, 224, 3)

In [ ]:
# KHỞI TẠO CÁC HÀM CẦN THIẾT (PHIÊN BẢN ĐÚNG)

def get_patient_id(filepath, class_name):
    filename = os.path.basename(filepath)
    if class_name.lower() in ['asthma', 'covid', 'healthy']:
        return filename.split('_')[0]
    elif class_name.lower() == 'tuberculosis':
        return '_'.join(filename.split('_')[:-1]).replace('.npy', '')
    else:
        return filename.split('_')[0]

class MacroF1Score(tf.keras.metrics.Metric):
    """
    Lớp metric để tính toán Macro F1-Score một cách chính xác trên toàn bộ epoch.
    """
    def __init__(self, num_classes, name='f1_macro', **kwargs):
        super(MacroF1Score, self).__init__(name=name, **kwargs)
        self.num_classes = num_classes
        # Tạo các biến để lưu trữ các giá trị TP, FP, FN qua các batch
        self.true_positives = self.add_weight(name='tp', shape=(num_classes,), initializer='zeros')
        self.false_positives = self.add_weight(name='fp', shape=(num_classes,), initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', shape=(num_classes,), initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        # Chuyển đổi đầu ra của model (logits) và nhãn thật
        y_pred_labels = tf.argmax(tf.nn.softmax(y_pred), axis=1)
        y_true_labels = tf.argmax(y_true, axis=1)
        
        # Tính ma trận nhầm lẫn cho batch hiện tại
        cm = tf.math.confusion_matrix(y_true_labels, y_pred_labels, num_classes=self.num_classes, dtype=tf.float32)
        
        # Từ ma trận nhầm lẫn, tính TP, FP, FN cho mỗi lớp
        tp = tf.linalg.diag_part(cm)
        fp = tf.reduce_sum(cm, axis=0) - tp
        fn = tf.reduce_sum(cm, axis=1) - tp
        
        # Cập nhật các biến trạng thái
        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        # Tính toán Precision, Recall từ các giá trị đã tích lũy
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        
        # Tính F1-Score cho mỗi lớp
        f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
        
        # Lấy trung bình cộng (macro)
        macro_f1 = tf.reduce_mean(f1)
        return macro_f1

    def reset_state(self):
        # Reset các biến trạng thái về 0 ở đầu mỗi epoch
        self.true_positives.assign(tf.zeros(self.num_classes))
        self.false_positives.assign(tf.zeros(self.num_classes))
        self.false_negatives.assign(tf.zeros(self.num_classes))
def parse_tfrecord_fn(example):
    """Hàm đọc và xử lý một mẫu từ file TFRecord cho EfficientNet."""
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.int64),
    }
    example = tf.io.parse_single_example(example, feature_description)
    image = tf.io.parse_tensor(example['image'], out_type=tf.float32)
    image = tf.reshape(image, (256, 126))
    
    # --- THAY ĐỔI QUAN TRỌNG ---
    # 1. Stack 3 kênh TRƯỚC KHI resize để tạo thành ảnh 3D
    image_3d = tf.stack([image, image, image], axis=-1)
    
    # 2. Bây giờ mới resize ảnh 3D này về kích thước mong muốn
    image_resized = tf.image.resize(image_3d, [224, 224])
    
    # 3. Sử dụng hàm tiền xử lý chuyên dụng của EfficientNet
    image_preprocessed = preprocess_input(image_resized)
    
    label_encoded = tf.cast(example['label'], tf.int32)
    label_onehot = tf.one_hot(label_encoded, depth=len(ALL_CLASSES))
    
    return image_preprocessed, label_onehot

def augment(spectrogram, label):
    spectrogram = spec_augment(spectrogram)
    return spectrogram, label

def focal_loss_from_logits_optimized(alpha, gamma=2.0):
    """
    Tạo ra hàm Focal Loss phiên bản đầy đủ và sạch sẽ.
    
    Args:
        alpha: Một list hoặc array chứa trọng số cho mỗi lớp.
        gamma: Hệ số tập trung, mặc định là 2.0.
    """
    # Chuyển alpha sang dạng tensor để tính toán
    alpha = tf.constant(alpha, dtype=tf.float32)

    def focal_loss_fixed(y_true, y_pred):
        y_true = tf.cast(y_true, 'float32')
        y_pred = tf.cast(y_pred, 'float32')
        
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_true, logits=y_pred)
        probs = tf.nn.softmax(y_pred)
        pt = tf.reduce_sum(y_true * probs, axis=-1)
        focal_term = (1.0 - pt) ** gamma
        alpha_t = tf.reduce_sum(y_true * alpha, axis=-1)
        loss = alpha_t * focal_term * cross_entropy
        
        return tf.reduce_mean(loss)
        
    return focal_loss_fixed

def spec_augment(spectrogram, time_masking_para=40, frequency_masking_para=30, num_time_masks=1, num_freq_masks=1):
    spectrogram_aug = spectrogram
    freq_bins = tf.shape(spectrogram)[1] # Sửa: Lấy chiều tần số từ shape 4D
    time_steps = tf.shape(spectrogram)[2] # Sửa: Lấy chiều thời gian từ shape 4D
    
    for _ in range(num_freq_masks):
        f = tf.random.uniform(shape=(), minval=0, maxval=frequency_masking_para, dtype=tf.int32)
        f0 = tf.random.uniform(shape=(), minval=0, maxval=freq_bins - f, dtype=tf.int32)
        freq_mask_1d = tf.concat([tf.ones((f0,), dtype=spectrogram.dtype), tf.zeros((f,), dtype=spectrogram.dtype), tf.ones((freq_bins - f0 - f,), dtype=spectrogram.dtype)], axis=0)
        freq_mask_4d = tf.reshape(freq_mask_1d, (1, freq_bins, 1, 1)) # Sửa: Reshape thành 4D để broadcast
        spectrogram_aug *= freq_mask_4d
        
    for _ in range(num_time_masks):
        t = tf.random.uniform(shape=(), minval=0, maxval=time_masking_para, dtype=tf.int32)
        t0 = tf.random.uniform(shape=(), minval=0, maxval=time_steps - t, dtype=tf.int32)
        time_mask_1d = tf.concat([tf.ones((t0,), dtype=spectrogram.dtype), tf.zeros((t,), dtype=spectrogram.dtype), tf.ones((time_steps - t0 - t,), dtype=spectrogram.dtype)], axis=0)
        time_mask_4d = tf.reshape(time_mask_1d, (1, 1, time_steps, 1)) # Sửa: Reshape thành 4D để broadcast
        spectrogram_aug *= time_mask_4d
        
    return spectrogram_aug

# HÀM CREATE_MODEL PHIÊN BẢN ĐÚNG VÀ ĐƠN GIẢN
class FinalModel(tf.keras.Model):
    def __init__(self, input_shape, num_classes):
        super(FinalModel, self).__init__()
        
        # --- SỬ DỤNG EfficientNetB0 LÀM MÔ HÌNH NỀN ---
        self.base_model = EfficientNetB0(
            weights='imagenet',      # Dùng trọng số tiền huấn luyện
            include_top=False,       # Bỏ lớp phân loại cuối cùng
            input_shape=input_shape
        )

        # Đóng băng các lớp như bình thường
        # (Bạn có thể cần thử nghiệm số lớp đóng băng cho EfficientNet)
        num_layers = len(self.base_model.layers)
        for layer in self.base_model.layers[:int(num_layers * 0.7)]: # Ví dụ: đóng băng 70%
             layer.trainable = False
        
        # Các lớp còn lại giữ nguyên
        self.pooling = GlobalAveragePooling2D()
        self.dropout = Dropout(0.5)
        self.dense_output = Dense(num_classes,
                                  activation='linear',
                                  kernel_regularizer=l2(0.001),
                                  dtype='float32')

    def call(self, inputs, training=None):
        x = self.base_model(inputs, training=training)
        x = self.pooling(x)
        x = self.dropout(x, training=training)
        outputs = self.dense_output(x)
        return outputs

def load_data_from_df(df):
    X, y = [], []
    for _, row in df.iterrows():
        X.append(np.load(row['filepath']))
        y.append(row['label'])
    return np.array(X), np.array(y)

def get_grad_cam_final(model, img_array, last_conv_layer_name, pred_index=None):
    global strategy
    
    with strategy.scope():
        grad_model = Model(
            [model.inputs[0]], 
            [model.get_layer(last_conv_layer_name).output, model.output]
        )

    with tf.GradientTape() as tape:
        # --- THAY ĐỔI QUAN TRỌNG ---
        # Thêm training=False để chỉ định rõ đây là chế độ inference
        last_conv_layer_output, preds = grad_model(tf.cast(img_array, tf.float32), training=False)
        
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    grads = tape.gradient(class_channel, last_conv_layer_output)

    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + tf.keras.backend.epsilon())
    
    return heatmap.numpy()

def overlay_grad_cam(spec, heatmap, alpha=0.6):
    heatmap_resized = tf.image.resize(heatmap[..., np.newaxis], (spec.shape[0], spec.shape[1]))
    heatmap_resized = np.uint8(255 * heatmap_resized)
    jet = plt.cm.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap_resized.squeeze()]
    spec_display = np.stack([spec]*3, axis=-1)
    spec_display = (spec_display - spec_display.min()) / (spec_display.max() - spec_display.min())
    superimposed_img = jet_heatmap * alpha + spec_display
    superimposed_img = np.clip(superimposed_img, 0, 1)
    return superimposed_img

class PDFReport(FPDF):
    def header(self):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, 'BAO CAO KET QUA HUAN LUYEN MO HINH AI', 0, 1, 'C')
        self.ln(10)
    def footer(self):
        self.set_y(-15)
        self.set_font('Arial', 'I', 8)
        self.cell(0, 10, f'Trang {self.page_no()}', 0, 0, 'C')
    def chapter_title(self, title):
        self.set_font('Arial', 'B', 12)
        self.cell(0, 10, title, 0, 1, 'L')
        self.ln(5)
    def chapter_body(self, content):
        self.set_font('Arial', '', 10)
        safe_content = content.encode('latin-1', 'replace').decode('latin-1')
        self.multi_cell(0, 5, safe_content)
        self.ln()
    def add_image_section(self, title, img_path):
        self.chapter_title(title)
        if os.path.exists(img_path):
            self.image(img_path, x=None, y=None, w=180)
            self.ln(5)
        else:
            self.chapter_body(f"Khong tim thay hinh anh: {img_path}")

def authenticate_gdrive():
    user_secrets = UserSecretsClient()
    secret_value = user_secrets.get_secret("google_service_account_key")
    with open("service_account.json", "w") as f:
        f.write(secret_value)
    scope = ["https://www.googleapis.com/auth/drive"]
    gauth = GoogleAuth()
    gauth.credentials = ServiceAccountCredentials.from_json_keyfile_name("service_account.json", scope)
    drive = GoogleDrive(gauth)
    return drive

def upload_folder_to_drive(drive, folder_path, parent_folder_id):
    folder_name = os.path.basename(folder_path)
    print(f"Đang tạo thư mục '{folder_name}' trên Google Drive...")
    folder_metadata = {'title': folder_name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [{'id': parent_folder_id}]}
    folder = drive.CreateFile(folder_metadata)
    folder.Upload()
    
    print(f"Bắt đầu tải nội dung của '{folder_name}'...")
    for item in tqdm(os.listdir(folder_path), desc=f"Uploading {folder_name}"):
        item_path = os.path.join(folder_path, item)
        if os.path.isfile(item_path):
            gfile = drive.CreateFile({'title': item, 'parents': [{'id': folder['id']}]})
            gfile.SetContentFile(item_path)
            gfile.Upload(param={'supportsTeamDrives': True})
        elif os.path.isdir(item_path):
            upload_folder_to_drive(drive, item_path, folder['id'])

def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy()
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(image, label):
    """Creates a tf.train.Example message ready to be written to a file."""
    feature = {
        'image': _bytes_feature(tf.io.serialize_tensor(image)),
        'label': _int64_feature(label)
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        current_lr = tf.keras.backend.get_value(self.model.optimizer.lr)
        print(f"\nEpoch {epoch+1}: Learning Rate is {current_lr:.2e}")

In [ ]:
# CHUẨN BỊ DỮ LIỆU VÀ TẠO TFRECORD
suspicious_files_to_remove = [
    '/kaggle/input/ngt-spectrogram-id/healthy/P0030101_123370_Dkwg3F7jMGaR7kbc-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0032202_15897_PcbyJQWemBfghUYp-seg2.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0027142_5701_hupBI5CxKMNCfe8b-seg1.npy',
    '/kaggle/input/ngt-spectrogram-id/covid/P0056214_89533_0WsmNRSKuQFGodg1-seg1.npy',
]
# --- BƯỚC 1: TẢI VÀ PHÂN CHIA DỮ LIỆU BAN ĐẦU ---
print("Bắt đầu chuẩn bị và phân chia dữ liệu...")
all_files_to_split = []
for class_name in ALL_CLASSES:
    source_dir = os.path.join(KAGGLE_PROCESSED_DATA_PATH, class_name)
    if os.path.exists(source_dir):
        files = glob.glob(os.path.join(source_dir, '*.npy'))
        for f in files:
            all_files_to_split.append({'filepath': f, 'label': class_name})

all_data_df = pd.DataFrame(all_files_to_split)
all_data_df['patient_id'] = all_data_df.apply(lambda row: get_patient_id(row['filepath'], row['label']), axis=1)

print(f"Số lượng mẫu ban đầu: {len(all_data_df)}")
all_data_df = all_data_df[~all_data_df['filepath'].isin(suspicious_files_to_remove)].reset_index(drop=True)
print(f"Số lượng mẫu sau khi lọc bỏ file 'im lặng': {len(all_data_df)}")

print("Tách tập Test cuối cùng (Hold-out set)...")
patient_ids = all_data_df['patient_id'].unique()
np.random.shuffle(patient_ids)
test_patient_count = int(len(patient_ids) * TEST_SPLIT_RATIO)
test_patients = patient_ids[:test_patient_count]
train_val_patients = patient_ids[test_patient_count:]

test_df = all_data_df[all_data_df['patient_id'].isin(test_patients)].reset_index(drop=True)
train_val_df = all_data_df[all_data_df['patient_id'].isin(train_val_patients)].reset_index(drop=True)

print(f"Đã tách: {len(train_val_df)} mẫu cho Train/Validation (CV) và {len(test_df)} mẫu cho Test cuối cùng.")

# --- BƯỚC 2: KHỞI TẠO LABEL ENCODER VÀ STANDARD SCALER ---
le = LabelEncoder().fit(ALL_CLASSES)
# Cập nhật INPUT_SHAPE từ một file mẫu
sample_spec = np.load(train_val_df['filepath'][0])
print(f"Kích thước input được cập nhật: {INPUT_SHAPE}")


# --- BƯỚC 3: CHUYỂN ĐỔI DỮ LIỆU SANG ĐỊNH DẠNG TFRECORD ---
TFRECORD_OUTPUT_PATH = "/kaggle/working/tfrecords"
os.makedirs(TFRECORD_OUTPUT_PATH, exist_ok=True)
print(f"Bắt đầu chuyển đổi dữ liệu sang TFRecord tại: {TFRECORD_OUTPUT_PATH}")

all_dfs = {'train_val': train_val_df, 'test': test_df}

for df_name, df in all_dfs.items():
    print(f"--- Đang xử lý tập {df_name} ---")
    tfrecord_path = os.path.join(TFRECORD_OUTPUT_PATH, f"{df_name}.tfrec")
    
    with tf.io.TFRecordWriter(tfrecord_path) as writer:
        # Sử dụng tqdm tiêu chuẩn để tránh lỗi ImportError
        for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Creating {df_name}.tfrec"):
            spectrogram = np.load(row['filepath']).astype(np.float32)
            label_encoded = le.transform([row['label']])[0]
            
            example = serialize_example(spectrogram, label_encoded)
            writer.write(example)
            
print("\nChuyển đổi sang TFRecord hoàn tất!")

In [ ]:
# HUẤN LUYỆN MÔ HÌNH Cross-Validation

# --- BƯỚC 1: KHỞI TẠO CÁC BIẾN CẦN THIẾT ---
print("Đang khởi tạo các biến cho Cross-Validation...")
AUTOTUNE = tf.data.AUTOTUNE
skf = StratifiedGroupKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
cv_data_to_split = train_val_df[train_val_df['label'].isin(CLASSES_TO_TRAIN)]
X_cv_paths = cv_data_to_split['filepath'].values
y_cv_labels = le.transform(cv_data_to_split['label'])
groups_cv = cv_data_to_split['patient_id'].values

fold_accuracies, fold_losses, fold_aucs, fold_f1s = [], [], [], []

print("Đang tính toán trọng số alpha cho Focal Loss...")
class_weights_array = class_weight.compute_class_weight('balanced', classes=np.unique(y_cv_labels), y=y_cv_labels)
alpha_weights_list = class_weights_array.tolist()
print("Trọng số Alpha được tính toán:")
for i, w in enumerate(alpha_weights_list):
    class_name = le.inverse_transform([i])[0]
    print(f"- Lớp '{class_name}': {w:.2f}")

# --- BƯỚC 2: BẮT ĐẦU VÒNG LẶP CROSS-VALIDATION ---
LOCAL_TFRECORD_PATH = TFRECORD_OUTPUT_PATH
TRAIN_VAL_TFREC = os.path.join(LOCAL_TFRECORD_PATH, 'train_val.tfrec')
print(f"Sẵn sàng đọc dữ liệu TFRecord từ: {TRAIN_VAL_TFREC}")

for fold, (train_indices, val_indices) in enumerate(skf.split(X_cv_paths, y_cv_labels, groups_cv)):
    fold_number = fold + 1
    print("-" * 50 + f"\nBắt đầu Fold {fold_number}/{N_SPLITS}\n" + "-" * 50)
    # TÍNH TOÁN CÁC BƯỚC CHO SCHEDULER
    # 1. Tạo pipeline dữ liệu trước
    train_indices_tf = tf.constant(train_indices, dtype=tf.int64)
    val_indices_tf = tf.constant(val_indices, dtype=tf.int64)
    full_ds = tf.data.TFRecordDataset(TRAIN_VAL_TFREC).enumerate()
    train_ds = full_ds.filter(lambda i, data: tf.reduce_any(i == train_indices_tf)).map(lambda i, data: data)
    val_ds = full_ds.filter(lambda i, data: tf.reduce_any(i == val_indices_tf)).map(lambda i, data: data)
    train_ds = train_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE).repeat().batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().batch(GLOBAL_BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
    
    # 2. Bây giờ mới tính toán steps_per_epoch
    steps_per_epoch = len(train_indices) // GLOBAL_BATCH_SIZE
    validation_steps = len(val_indices) // GLOBAL_BATCH_SIZE
    print(f"Số bước mỗi epoch: {steps_per_epoch} | Số bước validation: {validation_steps}")


    # --- TẠO MODEL VÀ LOSS FUNCTION ---
    with strategy.scope():
        model = FinalModel(input_shape=INPUT_SHAPE, num_classes=len(ALL_CLASSES))
        if USE_FOCAL_LOSS:
            loss_function = focal_loss_from_logits_optimized(alpha=alpha_weights_list, gamma=GAMMA)
        else:
            loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)

    # --- GIAI ĐOẠN 1: HUẤN LUYỆN CÁC LỚP CUỐI (HEAD TRAINING) ---
    print("\n--- Bắt đầu Giai đoạn 1: Huấn luyện các lớp cuối ---")
    with strategy.scope():
        # Đóng băng toàn bộ mô hình nền
        model.base_model.trainable = False
        # Compile với learning rate lớn hơn (ví dụ 1e-3)
        optimizer_head = tf.keras.optimizers.AdamW(learning_rate=1e-3, weight_decay=WEIGHT_DECAY)
        model.compile(optimizer=optimizer_head, loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])
        # Huấn luyện trong 5 epochs
        model.fit(train_ds, validation_data=val_ds, epochs=5,
              steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, verbose=1)

    # --- GIAI ĐOẠN 2: WARMUP VÀ HUẤN LUYỆN CHÍNH VỚI RESTART ---
    print("\n--- Bắt đầu Giai đoạn 2: Fine-tuning với Warmup & Restarts ---")

    # 1. THIẾT LẬP CÁC SIÊU THAM SỐ MỚI CHO LỊCH TRÌNH
    with strategy.scope():
        model.base_model.trainable = True
        f1_macro = MacroF1Score(num_classes=len(ALL_CLASSES), name='f1_macro')

        # 2. GIAI ĐOẠN 2A: WARMUP
        print(f"\n--- Giai đoạn 2A: Bắt đầu Warmup trong {WARMUP_EPOCHS} epochs ---")
        warmup_lr = LEARNING_RATE / 10 # Bắt đầu với LR rất thấp
        optimizer_warmup = tf.keras.optimizers.AdamW(learning_rate=warmup_lr, weight_decay=WEIGHT_DECAY)
        model.compile(optimizer=optimizer_warmup, loss=loss_function, metrics=['accuracy', f1_macro])

        model.fit(
            train_ds, validation_data=val_ds, epochs=WARMUP_EPOCHS,
            steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, verbose=1
        )

        # 3. GIAI ĐOẠN 2B: HUẤN LUYỆN CHÍNH VỚI COSINE DECAY RESTARTS
        # KHỞI TẠO COSINE DECAY SCHEDULER
        print(f"\n--- Giai đoạn 2B: Bắt đầu huấn luyện chính với CosineDecayRestarts ---")
        first_decay_steps = RESTART_CYCLE_1_EPOCHS * steps_per_epoch
        lr_scheduler = tf.keras.optimizers.schedules.CosineDecayRestarts(
            initial_learning_rate=LEARNING_RATE,
            first_decay_steps=first_decay_steps,
            t_mul=2.0, m_mul=0.9
        )
        optimizer_finetune = tf.keras.optimizers.AdamW(learning_rate=lr_scheduler, weight_decay=WEIGHT_DECAY)
        model.compile(optimizer=optimizer_finetune, loss=loss_function, metrics=['accuracy', tf.keras.metrics.AUC(name='auc'), f1_macro])
    
    callbacks = [
        EarlyStopping(
            monitor='val_f1_macro', mode='max', patience=PATIENCE_EPOCHS,
            restore_best_weights=True, min_delta=MIN_DELTA, verbose=1
        )
        LearningRateLogger()
    ]

    history = model.fit(
        train_ds, validation_data=val_ds, epochs=TOTAL_EPOCHS,
        initial_epoch=WARMUP_EPOCHS,
        steps_per_epoch=steps_per_epoch, validation_steps=validation_steps,
        callbacks=callbacks, verbose=1
    )
    # Tạo đường dẫn và tên file để lưu model
    model_save_path = os.path.join(KAGGLE_OUTPUT_PATH, f'{MODEL_ID}_fold_{fold_number}.keras')
    # Lưu lại toàn bộ mô hình (kiến trúc + trọng số)
    model.save(model_save_path)
    # In ra thông báo để xác nhận
    print(f"Đã lưu mô hình cho Fold {fold_number} tại: {model_save_path}")

    # --- VẼ BIỂU ĐỒ VÀ ĐÁNH GIÁ ---
    print("Đang tạo và lưu biểu đồ huấn luyện...")
    plt.figure(figsize=(18, 7))
    plt.suptitle(f'Training Metrics for Fold {fold_number}', fontsize=16)
    
    # --- Biểu đồ cho các chỉ số (Accuracy, AUC, F1-Macro) ---
    plt.subplot(1, 2, 1)
    # Vẽ các chỉ số của tập Train
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='blue', linestyle='-')
    plt.plot(history.history['auc'], label='Training AUC', color='green', linestyle='-')
    plt.plot(history.history['f1_macro'], label='Training F1-Macro', color='red', linestyle='-')
    
    # Vẽ các chỉ số của tập Validation
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='blue', linestyle='--')
    plt.plot(history.history['val_auc'], label='Validation AUC', color='green', linestyle='--')
    plt.plot(history.history['val_f1_macro'], label='Validation F1-Macro', color='red', linestyle='--')
    
    # Cập nhật lại tiêu đề và nhãn
    plt.title('Biểu đồ các chỉ số (Accuracy, AUC, F1-Macro)')
    plt.xlabel('Epoch')
    plt.ylabel('Giá trị')
    plt.legend(loc='lower right')
    plt.grid(True)
    
    # --- Biểu đồ cho Loss ---
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='orange')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='purple')
    plt.title('Biểu đồ Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    
    # Lưu và đóng hình ảnh
    plot_filename = f'fold_{fold_number}_metrics.png'
    plot_filepath = os.path.join(KAGGLE_OUTPUT_PATH, plot_filename)
    plt.savefig(plot_filepath)
    plt.close()
    
    print(f"Đã lưu biểu đồ cho Fold {fold_number} tại: {plot_filepath}")
    
    # THÊM 4: Sửa lại model.evaluate để nhận đủ 4 giá trị
    loss, accuracy, auc, f1 = model.evaluate(val_ds, verbose=0)
    print(f"Fold {fold_number} - Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}, Validation AUC: {auc:.4f}, Validation F1-Macro: {f1:.4f}")
    
    fold_aucs.append(auc)
    fold_losses.append(loss)
    fold_accuracies.append(accuracy)
    fold_f1s.append(f1) # Thêm lưu trữ F1

    # THÊM 5: Sửa lại câu lệnh print cuối cùng
    print("=" * 50 + "\nKết quả Cross-Validation:\n" 
      + f"Validation Accuracy trung bình: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}\n"
      + f"Validation Loss trung bình: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}\n"
      + f"Validation AUC trung bình: {np.mean(fold_aucs):.4f} +/- {np.std(fold_aucs):.4f}\n"
      + f"Validation F1-Macro trung bình: {np.mean(fold_f1s):.4f} +/- {np.std(fold_f1s):.4f}\n" # Thêm dòng này
      + "=" * 50)

In [ ]:
# ĐÁNH GIÁ MÔ HÌNH VÀ VẼ CÁC SƠ ĐỒ (PHIÊN BẢN HOÀN CHỈNH)
print("\nĐang đánh giá mô hình cuối cùng trên tập Test (Hold-out)...")

# --- BƯỚC 1: TẠO KIẾN TRÚC, TẢI TRỌNG SỐ VÀ BIÊN DỊCH LẠI MODEL ---

print("Đang tạo lại kiến trúc model và tải trọng số đã lưu...")
with strategy.scope():
    # Tạo một kiến trúc model mới y hệt lúc huấn luyện
    final_model = FinalModel(input_shape=INPUT_SHAPE, num_classes=len(ALL_CLASSES))
    # THÊM BƯỚC BUILD: Xây dựng model với input shape đã biết
    # (None, *INPUT_SHAPE) để bao gồm cả chiều batch (batch dimension)
    final_model.build(input_shape=(None, *INPUT_SHAPE))

    # Bây giờ mới nạp trọng số
    final_model.load_weights(model_checkpoint_path)
    
    # Tải các trọng số đã được huấn luyện vào kiến trúc mới này
    final_model.load_weights(model_checkpoint_path)

    # --- THÊM VÀO ĐÂY: BIÊN DỊCH LẠI MODEL ---
    # Model cần được compile với đúng các thiết lập như khi huấn luyện
    # để có thể tính toán loss và metrics.
    print("Đang biên dịch lại model...")
    if USE_FOCAL_LOSS:
        loss_function = focal_loss_from_logits_optimized(alpha=final_alpha_weights_list, gamma=GAMMA)
    else:
        loss_function = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
        
    final_optimizer = tf.keras.optimizers.AdamW(learning_rate=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    
    final_model.compile(optimizer=final_optimizer, 
                        loss=loss_function, 
                        metrics=['accuracy', tf.keras.metrics.AUC(name='auc')])

print("Tải và biên dịch model cuối cùng hoàn tất.")


# --- BƯỚC 2: TẠO PIPELINE DỮ LIỆU CHO TẬP TEST ---
TEST_TFREC = os.path.join(LOCAL_TFRECORD_PATH, 'test.tfrec')
print(f"Sử dụng dữ liệu test từ: {TEST_TFREC}")

test_ds = tf.data.TFRecordDataset(TEST_TFREC)
test_ds = test_ds.map(parse_tfrecord_fn, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.batch(GLOBAL_BATCH_SIZE, drop_remainder=True)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)


# --- BƯỚC 3: ĐÁNH GIÁ VÀ DỰ ĐOÁN TRÊN PIPELINE ---
print("Đang đánh giá trên tập test...")
loss, accuracy, auc = final_model.evaluate(test_ds, verbose=1)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}, Test AUC: {auc:.4f}")

print("Đang dự đoán trên tập test...")
y_pred_logits = final_model.predict(test_ds, verbose=1) # Đổi tên thành logits để rõ ràng hơn
y_pred_encoded = np.argmax(y_pred_logits, axis=1)

# Lấy nhãn thật từ dataframe
final_test_df = test_df[test_df['label'].isin(CLASSES_TO_TRAIN)]
y_test_labels = final_test_df['label'].values
y_test_encoded = le.transform(y_test_labels)
y_test_encoded = y_test_encoded[:len(y_pred_encoded)]

trained_class_indices = np.unique(y_test_encoded)
target_names_trained = le.inverse_transform(trained_class_indices)


# --- BƯỚC 4: TÍNH TOÁN VÀ TRÍCH XUẤT CÁC CHỈ SỐ CHI TIẾT ---
# Sử dụng output_dict=True để lấy kết quả dưới dạng dictionary
report_dict = classification_report(y_test_encoded, y_pred_encoded, target_names=target_names_trained, labels=trained_class_indices, output_dict=True)
report_text = classification_report(y_test_encoded, y_pred_encoded, target_names=target_names_trained, labels=trained_class_indices)

print("\nClassification Report:\n", report_text)

# Ví dụ cách trích xuất các chỉ số bạn cần cho báo cáo:
macro_f1 = report_dict['macro avg']['f1-score']
weighted_f1 = report_dict['weighted avg']['f1-score']
asthma_recall = report_dict['asthma']['recall']

print(f"\\nCác chỉ số riêng lẻ:")
print(f"- Macro F1-Score: {macro_f1:.4f}")
print(f"- Weighted F1-Score: {weighted_f1:.4f}")
print(f"- Recall của lớp 'asthma': {asthma_recall:.4f}")


# --- BƯỚC 5: VẼ CÁC BIỂU ĐỒ ---
report_figs_path = os.path.join(KAGGLE_OUTPUT_PATH, "report_figures")
os.makedirs(report_figs_path, exist_ok=True)

# Biểu đồ training history (giữ nguyên)
plt.figure(figsize=(8, 6))
plt.plot(final_history.history['accuracy'], label='Training Accuracy')
plt.plot(final_history.history['auc'], label='Training AUC') # Thêm AUC vào biểu đồ
plt.title('Biểu đồ Accuracy & AUC của mô hình cuối cùng')
plt.xlabel('Epoch')
plt.ylabel('Value')
plt.legend(loc='lower right')
accuracy_plot_path = os.path.join(report_figs_path, f'final_accuracy_plot_{run_timestamp}.png')
plt.savefig(accuracy_plot_path)
plt.close()

plt.figure(figsize=(8, 6))
plt.plot(final_history.history['loss'], label='Training Loss')
plt.title('Biểu đồ Loss của mô hình cuối cùng')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
loss_plot_path = os.path.join(report_figs_path, f'final_loss_plot_{run_timestamp}.png')
plt.savefig(loss_plot_path)
plt.close()

# Ma trận nhầm lẫn (giữ nguyên)
cm = confusion_matrix(y_test_encoded, y_pred_encoded, labels=trained_class_indices)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names_trained, yticklabels=target_names_trained)
plt.title('Ma trận nhầm lẫn trên tập Test cuối cùng')
plt.ylabel('Nhãn thật')
plt.xlabel('Nhãn dự đoán')
cm_plot_path = os.path.join(report_figs_path, f'confusion_matrix_{run_timestamp}.png')
plt.savefig(cm_plot_path)
plt.close()


# --- BƯỚC 6: TÍNH TOÁN VÀ VẼ ĐƯỜNG CONG ROC CHO TỪNG LỚP ---
# Chuyển logits thành xác suất
y_pred_probs = tf.nn.softmax(y_pred_logits).numpy()

# Chuyển nhãn thật sang dạng one-hot để tính ROC cho từng lớp
y_test_binarized = label_binarize(y_test_encoded, classes=trained_class_indices)
n_classes = y_test_binarized.shape[1]

# Tính toán ROC và AUC cho từng lớp
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_probs[:, i])
    roc_auc[i] = sklearn_auc(fpr[i], tpr[i])

# Vẽ đường cong ROC
plt.figure(figsize=(10, 8))
colors = cycle(['aqua', 'darkorange', 'cornflowerblue', 'deeppink'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='Đường cong ROC của lớp {0} (AUC = {1:0.2f})'
             ''.format(target_names_trained[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Tỷ lệ Dương tính Giả (False Positive Rate)')
plt.ylabel('Tỷ lệ Dương tính Thật (True Positive Rate)')
plt.title('Đường cong ROC cho từng lớp')
plt.legend(loc="lower right")
roc_plot_path = os.path.join(report_figs_path, f'roc_curves_{run_timestamp}.png')
plt.savefig(roc_plot_path)
plt.show()
plt.close()

In [ ]:
# VẼ GRAD-CAM (PHIÊN BẢN HOÀN CHỈNH)

print("Đang tạo lại kiến trúc model và tải trọng số đã lưu...")
with strategy.scope():
    # Tạo một kiến trúc model mới y hệt lúc huấn luyện
    final_model = create_model(INPUT_SHAPE, len(ALL_CLASSES))
    
    # Tải các trọng số đã được huấn luyện vào kiến trúc mới này
    final_model.load_weights(model_checkpoint_path)

print("Tải trọng số cho model cuối cùng hoàn tất.")

# --- BƯỚC 1: LẤY DỮ LIỆU TEST TỪ PIPELINE `test_ds` ---
print("Đang trích xuất dữ liệu từ test_ds để vẽ Grad-CAM...")
X_test_list = []
y_test_onehot_list = []

# Sử dụng tqdm để theo dõi tiến trình
for images, labels in tqdm(test_ds, desc="Extracting data from test_ds"):
    X_test_list.append(images.numpy())
    y_test_onehot_list.append(labels.numpy())

# Nối các batch lại thành các mảng NumPy lớn
X_test = np.concatenate(X_test_list, axis=0)
y_test_onehot = np.concatenate(y_test_onehot_list, axis=0)
y_test_encoded_from_ds = np.argmax(y_test_onehot, axis=1)

print(f"Đã trích xuất thành công {len(X_test)} mẫu.")


# --- BƯỚC 2: TÌM LỚP CONVOLUTION CUỐI CÙNG (giữ nguyên) ---
last_conv_layer_name = None
for layer in reversed(final_model.layers):
    if isinstance(layer, tf.keras.layers.GlobalAveragePooling2D):
        pooling_index = final_model.layers.index(layer)
        last_conv_layer_name = final_model.layers[pooling_index - 1].name
        break
if last_conv_layer_name is None:
    raise ValueError("Không thể tự động tìm thấy lớp phù hợp cho Grad-CAM.")
print(f"Đã tự động xác định lớp Grad-CAM: {last_conv_layer_name}")


# --- BƯỚC 3: TẠO HÌNH ẢNH GRAD-CAM CHO CÁC MẪU TIÊU BIỂU ---
gradcam_path = os.path.join(report_figs_path, "grad_cam")
os.makedirs(gradcam_path, exist_ok=True)
print("Tạo hình ảnh Grad-CAM cho các mẫu tiêu biểu...")

results_list = []
for i in range(len(y_pred_encoded)):
    true_label_encoded = y_test_encoded_from_ds[i]
    pred_label_encoded = y_pred_encoded[i]
    confidence = y_pred_probs[i][pred_label_encoded]
    is_correct = (true_label_encoded == pred_label_encoded)
    results_list.append({'index': i, 'true_label': true_label_encoded, 'pred_label': pred_label_encoded, 'confidence': confidence, 'is_correct': is_correct})
results_df = pd.DataFrame(results_list)

for class_index, class_name in zip(trained_class_indices, target_names_trained):
    correct_samples = results_df[(results_df['is_correct'] == True) & (results_df['true_label'] == class_index)].nlargest(3, 'confidence')
    incorrect_samples = results_df[(results_df['is_correct'] == False) & (results_df['true_label'] == class_index)].nlargest(3, 'confidence')
    
    for _, row in correct_samples.iterrows():
        idx = int(row['index'])
        img_array, spec = X_test[idx][np.newaxis, ...], X_test[idx, :, :, 0]
        # Đổi tên hàm thành get_grad_cam_final
        heatmap = get_grad_cam_final(final_model, img_array, last_conv_layer_name, pred_index=class_index)
        overlay = overlay_grad_cam(spec, heatmap)
        plt.imshow(overlay)
        plt.title(f"Đúng: {class_name}, Tin cậy: {row['confidence']:.2f}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"correct_{class_name}_{idx}_{run_timestamp}.png"))
        plt.close()
        
    for _, row in incorrect_samples.iterrows():
        idx = int(row['index'])
        pred_class_name = le.inverse_transform([int(row['pred_label'])])[0]
        img_array, spec = X_test[idx][np.newaxis, ...], X_test[idx, :, :, 0]
        # Đổi tên hàm thành get_grad_cam_final
        heatmap = get_grad_cam_final(final_model, img_array, last_conv_layer_name, pred_index=class_index)
        overlay = overlay_grad_cam(spec, heatmap)
        plt.imshow(overlay)
        plt.title(f"Thật: {class_name}, Sai -> {pred_class_name}, Tin cậy: {row['confidence']:.2f}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"incorrect_{class_name}_as_{pred_class_name}_{idx}_{run_timestamp}.png"))
        plt.close()

# --- BƯỚC 4: TÍNH TOÁN VÀ VẼ GRAD-CAM TRUNG BÌNH ---
print("Đang tính toán Grad-CAM trung bình cho tất cả các lớp...")

# Tạo các dictionary để lưu trữ heatmaps
correct_heatmaps = {class_name: [] for class_name in target_names_trained}
incorrect_heatmaps = {class_name: [] for class_name in target_names_trained}

# Lặp qua tất cả các kết quả để tính heatmap
for _, row in tqdm(results_df.iterrows(), total=len(results_df), desc="Calculating all heatmaps"):
    idx = int(row['index'])
    true_label_index = int(row['true_label'])
    true_class_name = le.inverse_transform([true_label_index])[0]
    
    img_array = X_test[idx][np.newaxis, ...]
    # Luôn tính heatmap theo nhãn thật để xem model tập trung vào đâu
    heatmap = get_grad_cam_final(final_model, img_array, last_conv_layer_name, pred_index=true_label_index)
    
    if row['is_correct']:
        correct_heatmaps[true_class_name].append(heatmap)
    else:
        incorrect_heatmaps[true_class_name].append(heatmap)

# Vẽ Grad-CAM trung bình
for class_name in target_names_trained:
    # Xử lý các ca đoán đúng
    if correct_heatmaps[class_name]:
        avg_heatmap_correct = np.mean(correct_heatmaps[class_name], axis=0)
        overlay = overlay_grad_cam(np.zeros(INPUT_SHAPE[:2]), avg_heatmap_correct)
        plt.imshow(overlay)
        plt.title(f"Grad-CAM TB - Đúng cho lớp {class_name}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"avg_correct_{class_name}_{run_timestamp}.png"))
        plt.close()
        
    # Xử lý các ca đoán sai
    if incorrect_heatmaps[class_name]:
        avg_heatmap_incorrect = np.mean(incorrect_heatmaps[class_name], axis=0)
        overlay = overlay_grad_cam(np.zeros(INPUT_SHAPE[:2]), avg_heatmap_incorrect)
        plt.imshow(overlay)
        plt.title(f"Grad-CAM TB - Sai cho lớp {class_name}")
        plt.axis('off')
        plt.savefig(os.path.join(gradcam_path, f"avg_incorrect_{class_name}_{run_timestamp}.png"))
        plt.close()

print("Hoàn tất việc tạo Grad-CAM.")

In [ ]:
# TẠO BÁO CÁO PDF
print("Tạo báo cáo PDF...")
pdf = PDFReport()
pdf.add_page()
pdf.chapter_title("1. Tom tat cau hinh va Ket qua")
config_summary = f"""
- Model ID: {MODEL_ID}
- Thoi gian chay: {datetime.datetime.now(pytz.timezone('Asia/Ho_Chi_Minh')).strftime("%Y-%m-%d %H:%M:%S")}
- Cac lop huan luyen: {', '.join(CLASSES_TO_TRAIN)}
- K-Fold Cross-Validation: {N_SPLITS} folds

--- KET QUA CROSS-VALIDATION ---
- Validation Accuracy trung binh: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}
- Validation Loss trung binh: {np.mean(fold_losses):.4f} +/- {np.std(fold_losses):.4f}

--- KET QUA TREN TAP TEST CUOI CUNG ---
- Test Loss: {loss:.4f}
- Test Accuracy: {accuracy:.4f}

--- CAU HINH CHI TIET ---
- SEED: {SEED}
- Epochs: {EPOCHS} (Patience: {EARLY_STOPPING_PATIENCE})
- Batch Size: {BATCH_SIZE}
- Learning Rate: {LEARNING_RATE}
- Ham Loss: {'Focal Loss' if USE_FOCAL_LOSS else 'Categorical Crossentropy'}
- Tang cuong du lieu: {'Co (SpecAugment)' if USE_DATA_AUGMENTATION else 'Khong'}
- Kich thuoc Input: {INPUT_SHAPE}
"""
pdf.chapter_body(config_summary)
pdf.add_image_section("2. Bieu do Huan luyen cua Mo hinh Cuoi cung", accuracy_plot_path)
pdf.add_image_section("", loss_plot_path)
pdf.chapter_title("3. Danh gia chi tiet tren tap Test")
pdf.chapter_body("Bao cao phan loai chi tiet:")
pdf.set_font('Courier', '', 8)
pdf.chapter_body(report)
pdf.add_image_section("Ma tran nham lan:", cm_plot_path)

pdf.add_page()
pdf.chapter_title("4. Phan tich Grad-CAM")
for class_name in target_names_trained:
    pdf.chapter_body(f"Lop: {class_name}")
    correct_imgs = sorted(glob.glob(os.path.join(gradcam_path, f"correct_{class_name}_*_{run_timestamp}.png")))
    incorrect_imgs = sorted(glob.glob(os.path.join(gradcam_path, f"incorrect_{class_name}_*_{run_timestamp}.png")))
    
    x_pos, y_pos = pdf.get_x(), pdf.get_y()
    for i, img_path in enumerate(correct_imgs[:3]):
        if os.path.exists(img_path): pdf.image(img_path, x=x_pos + i * 60, y=y_pos, w=55)
    if correct_imgs: y_pos += 45
    for i, img_path in enumerate(incorrect_imgs[:3]):
        if os.path.exists(img_path): pdf.image(img_path, x=x_pos + i * 60, y=y_pos, w=55)
    if incorrect_imgs: y_pos += 45
    pdf.set_y(y_pos)
    
    avg_correct_path = os.path.join(gradcam_path, f"avg_correct_{class_name}_{run_timestamp}.png")
    avg_incorrect_path = os.path.join(gradcam_path, f"avg_incorrect_{class_name}_{run_timestamp}.png")
    if os.path.exists(avg_correct_path):
        pdf.image(avg_correct_path, w=80)
    if os.path.exists(avg_incorrect_path):
        pdf.image(avg_incorrect_path, w=80)
    pdf.ln(10)

report_filename = f"report_{MODEL_ID}_{run_timestamp}.pdf"
report_filepath = os.path.join(KAGGLE_OUTPUT_PATH, report_filename)
pdf.output(report_filepath)
print(f"Đã tạo báo cáo PDF tại: {report_filepath}")

print("\nBắt đầu quá trình tải kết quả lên Google Drive...")
drive = authenticate_gdrive()
upload_folder_to_drive(drive, KAGGLE_OUTPUT_PATH, DRIVE_RESULTS_FOLDER_ID)
print("Hoàn tất! Toàn bộ kết quả đã được lưu về Google Drive.")